In [ ]:
# File for regression

In [1]:
from os.path import join
from google.colab import drive

# path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = 'My Drive/Capstone Project'
ROOT = '/content/gdrive/'
drive.mount(ROOT)

Mounted at /content/gdrive/


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [ ]:
#Setting Random Seed
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
#Setting Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
data_dir = ROOT + MY_GOOGLE_DRIVE_PATH + '/Edge/data/'

In [ ]:
mv = pd.read_table(data_dir + 'mv.dat',skiprows=15,delimiter=",",header=None)
mv.columns = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 
                     'X7', 'X8', 'X9', 'X10', 'Y']

california = pd.read_table(data_dir + 'california.dat',skiprows=13,delimiter=",",header=None)
california.columns = ['Longitude', 'Latitude', 'HousingMedianAge', 'TotalRooms', 'TotalBedrooms', 'Population', 
                     'Households', 'MedianIncome', 'MedianHouseValue']

print('Shape - MV: {} & California: {}'.format(mv.shape, california.shape))

Shape - MV: (40768, 11) & California: (20640, 9)


In [ ]:
#Model Architecture
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size[0]) 
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])  
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size[1], num_classes)  

    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)

        return out

In [ ]:
 l =[10,20,30]
print(l.append([2]))

None


In [ ]:
class DenseNeuralNet(nn.Module):
  """ 
  Class to create the architecture of dense neural network and the forward pass of batch 
  """
  def __init__(self, input_size, num_classes,layers,dropout_prob = 0.0, batch_norm = False):
    super().__init__()
    layers.append(num_classes)
    config = {
        "num_layers" : len(layers),
        "nodes_per_layer" : layers,
        "dropout_prob" : dropout_prob,
        "batch_norm" : batch_norm
    }

    layers = []
    prev = input_size
  
    for i in range(0, config["num_layers"]):
      layers.append(nn.Linear(prev, config["nodes_per_layer"][i]))
      if not i==config["num_layers"]-1:
        if config["batch_norm"]:
          layers.append(nn.BatchNorm1d(num_features = config["nodes_per_layer"][i]))
        layers.append(nn.ReLU())

        if config["dropout_prob"] > 0:
          layers.append(nn.Dropout(p = config["dropout_prob"]))

      prev = config["nodes_per_layer"][i]
    
    self.net = nn.Sequential(*layers)

    
  def forward(self, x):
      out = self.net(x)
      return out

In [ ]:
mv.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,Y
0,-4.809150,-10.5062,0,-5.25312,0.665394,-23.62270,0,1,445.012,1082,-2.40458
1,-0.928705,-10.8176,1,-5.40878,0.430141,-25.32900,0,0,250.038,1115,-26.25770
2,1.304560,-11.9609,0,-5.98045,0.815231,-7.46146,0,1,261.694,1163,0.65228
3,-4.068760,-11.7433,0,-5.87165,0.285156,-2.89871,1,0,175.301,1191,-6.96747
4,4.750250,-12.9444,2,-6.47220,0.701619,-12.84180,1,1,341.723,1010,2.37512


In [ ]:
mv_scaler = MinMaxScaler()

In [ ]:
data = mv_scaler.fit_transform(mv.iloc[:, :-1])
target = np.asarray(mv.Y)

data = data.astype(np.float32)
target = target.astype(np.float32)

mv = {'data': torch.from_numpy(data), 'target': torch.from_numpy(target).unsqueeze(1)}

In [ ]:
mv_dataset = torch.utils.data.TensorDataset(mv["data"],mv["target"])

In [ ]:
#hyperparameters
input_size = mv["data"].shape[1]
hidden_size = [10,20]
num_classes = 1
num_epochs = 10
batch_size = 16
learning_rate = 0.001

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=mv_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True,
                                           drop_last=True)

In [ ]:
model_mv = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_mv.parameters(), lr=learning_rate)  

In [ ]:
print(model_mv)

NeuralNet(
  (fc1): Linear(in_features=10, out_features=10, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=10, out_features=20, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=20, out_features=1, bias=True)
)


In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        features = features.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_mv(features)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 1000 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/10], Step [1000/2548], Loss: 38.5926
Epoch [1/10], Step [2000/2548], Loss: 0.9406
Epoch [2/10], Step [1000/2548], Loss: 0.3360
Epoch [2/10], Step [2000/2548], Loss: 0.4871
Epoch [3/10], Step [1000/2548], Loss: 0.0938
Epoch [3/10], Step [2000/2548], Loss: 0.4136
Epoch [4/10], Step [1000/2548], Loss: 0.0485
Epoch [4/10], Step [2000/2548], Loss: 0.0331
Epoch [5/10], Step [1000/2548], Loss: 0.0109
Epoch [5/10], Step [2000/2548], Loss: 0.0160
Epoch [6/10], Step [1000/2548], Loss: 0.0087
Epoch [6/10], Step [2000/2548], Loss: 0.0930
Epoch [7/10], Step [1000/2548], Loss: 0.0165
Epoch [7/10], Step [2000/2548], Loss: 0.0092
Epoch [8/10], Step [1000/2548], Loss: 0.0026
Epoch [8/10], Step [2000/2548], Loss: 0.0093
Epoch [9/10], Step [1000/2548], Loss: 0.0040
Epoch [9/10], Step [2000/2548], Loss: 0.0072
Epoch [10/10], Step [1000/2548], Loss: 0.0065
Epoch [10/10], Step [2000/2548], Loss: 0.0487


In [ ]:
l = [module for module in model_mv.children() if type(module) != nn.Sequential]

In [ ]:
for i in l:
  try:
    print('Layer: {} weight:{}'.format(i,i.weight))
  except Exception as e: 
    print(e)

Layer: Linear(in_features=10, out_features=10, bias=True) weight:Parameter containing:
tensor([[ 0.4699,  0.3205,  0.0964, -0.3517,  0.1180,  1.0426,  0.0946,  1.4341,
          0.1516,  0.1717],
        [-0.1785,  0.0047,  0.0041, -0.2867, -0.0121, -1.0999, -0.0937, -0.9806,
          0.0129,  0.0258],
        [ 0.9145,  0.3340,  0.1180,  0.6930,  0.2060, -0.6327,  0.3430, -1.0131,
          0.1236,  0.1308],
        [-0.1834,  0.1002,  0.0615, -0.0669,  0.0487, -1.1396,  0.0271, -1.1309,
          0.0651,  0.0698],
        [ 0.0166, -0.1621,  0.0535, -0.2953, -0.2285, -0.1630,  0.1995,  0.1854,
         -0.1402, -0.0114],
        [ 0.2588,  0.0282,  0.0062, -0.5826,  0.0813, -0.8060, -0.1056, -1.2478,
          0.0286,  0.0172],
        [ 1.1023,  0.1026,  0.1699, -0.0478,  0.2562, -1.0915, -0.0260, -0.8900,
          0.1768,  0.1095],
        [ 0.6601,  0.2524,  0.2908,  0.5590,  0.4568,  1.4820,  0.2204,  0.5543,
          0.2703,  0.1980],
        [-0.2974,  0.4506,  0.2902,  0.10

In [ ]:
print(l[0].weight.cpu().detach().numpy())

[[ 0.46989742  0.32048604  0.09638401 -0.35171282  0.11802114  1.0425825
   0.09461611  1.434067    0.15157722  0.17171012]
 [-0.17852172  0.0047252   0.00407568 -0.2867082  -0.01209975 -1.0998722
  -0.09369025 -0.9806231   0.01286327  0.02583187]
 [ 0.91452605  0.33395085  0.11799423  0.6930265   0.20595504 -0.6326861
   0.3430459  -1.0130923   0.12363158  0.13083911]
 [-0.18339717  0.100206    0.06146965 -0.06691787  0.04870877 -1.1395509
   0.02707295 -1.1309048   0.06509991  0.06980738]
 [ 0.01663091 -0.16212475  0.05350089 -0.2952602  -0.22849551 -0.1630249
   0.1995201   0.18541105 -0.14024544 -0.01141025]
 [ 0.25876915  0.02815503  0.00617451 -0.5826318   0.08129841 -0.806013
  -0.1056397  -1.2477994   0.02856905  0.01715771]
 [ 1.1022768   0.10259777  0.169925   -0.04775922  0.25618905 -1.0915233
  -0.02600583 -0.89000267  0.17684951  0.10947178]
 [ 0.6600744   0.25241274  0.29081148  0.55904835  0.4568133   1.4820105
   0.2203751   0.55434537  0.2703183   0.1979986 ]
 [-0.2973

In [ ]:
with torch.no_grad():
  for i in l:
    try:
      i.weight =  torch.nn.Parameter( (i.weight).round()).to(device)
      # print('Layer: {} weight:{}'.format(i,i.weight))
    except Exception as e: 
      print(e)

'ReLU' object has no attribute 'weight'
'ReLU' object has no attribute 'weight'


In [ ]:
for i in l:
  try:
    # i.weight =  torch.nn.Parameter( (i.weight).round()).to(device)
    print('Layer: {} weight:{}'.format(i,i.weight))
  except Exception as e: 
    print(e)

Layer: Linear(in_features=10, out_features=10, bias=True) weight:Parameter containing:
tensor([[ 0.,  0.,  0., -0.,  0.,  1.,  0.,  1.,  0.,  0.],
        [-0.,  0.,  0., -0., -0., -1., -0., -1.,  0.,  0.],
        [ 1.,  0.,  0.,  1.,  0., -1.,  0., -1.,  0.,  0.],
        [-0.,  0.,  0., -0.,  0., -1.,  0., -1.,  0.,  0.],
        [ 0., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0.],
        [ 0.,  0.,  0., -1.,  0., -1., -0., -1.,  0.,  0.],
        [ 1.,  0.,  0., -0.,  0., -1., -0., -1.,  0.,  0.],
        [ 1.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.],
        [-0.,  0.,  0.,  0.,  1., -0.,  0., -1.,  0.,  0.],
        [-0.,  0.,  0.,  1.,  0., -0.,  0.,  1.,  0.,  0.]], device='cuda:0',
       requires_grad=True)
'ReLU' object has no attribute 'weight'
Layer: Linear(in_features=10, out_features=20, bias=True) weight:Parameter containing:
tensor([[-0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.],
        [ 0., -0., -0.,  0., -0., -0., -0., -0., -0., -0.],
        [-0.,  0., -0

In [ ]:
input_dim =  mv["data"].shape[1]
output_classes = 1
learning_rate = 0.001
batch_size = 16
num_epochs = 10
####
train_loader = torch.utils.data.DataLoader(dataset=mv_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True,
                                           drop_last=True)


In [ ]:
model_mv1 = DenseNeuralNet(input_dim, output_classes,layers=[50,100,50]).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_mv1.parameters(), lr=learning_rate)  

In [ ]:
print(model_mv1)

DenseNeuralNet(
  (net): Sequential(
    (0): Linear(in_features=10, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=50, bias=True)
    (5): ReLU()
    (6): Linear(in_features=50, out_features=1, bias=True)
  )
)


In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        features = features.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_mv1(features)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 1000 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/10], Step [1000/2548], Loss: 0.2761
Epoch [1/10], Step [2000/2548], Loss: 0.0261
Epoch [2/10], Step [1000/2548], Loss: 0.0072
Epoch [2/10], Step [2000/2548], Loss: 0.0224
Epoch [3/10], Step [1000/2548], Loss: 0.0235
Epoch [3/10], Step [2000/2548], Loss: 0.0032
Epoch [4/10], Step [1000/2548], Loss: 0.0206
Epoch [4/10], Step [2000/2548], Loss: 0.0154
Epoch [5/10], Step [1000/2548], Loss: 0.0182
Epoch [5/10], Step [2000/2548], Loss: 0.0037
Epoch [6/10], Step [1000/2548], Loss: 0.0008
Epoch [6/10], Step [2000/2548], Loss: 0.0019
Epoch [7/10], Step [1000/2548], Loss: 0.0079
Epoch [7/10], Step [2000/2548], Loss: 0.0165
Epoch [8/10], Step [1000/2548], Loss: 0.0034
Epoch [8/10], Step [2000/2548], Loss: 0.0024
Epoch [9/10], Step [1000/2548], Loss: 0.0020
Epoch [9/10], Step [2000/2548], Loss: 0.0069
Epoch [10/10], Step [1000/2548], Loss: 0.0090
Epoch [10/10], Step [2000/2548], Loss: 0.0010


In [ ]:
california.MedianHouseValue/=(1000)
california.describe()

,Longitude,Latitude,HousingMedianAge,TotalRooms,TotalBedrooms,Population,Households,MedianIncome,MedianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206.855817
std,2.003532,2.135952,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115.395616
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14.999000
25%,-121.800000,33.930000,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119.600000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179.700000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264.725000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500.001000


In [ ]:
low = .05
high = .95
quant_california = california.MedianHouseValue.quantile([low, high])
print(quant_california)

0.05     66.20
0.95    489.81
Name: MedianHouseValue, dtype: float64


In [ ]:
temp = california[(california.MedianHouseValue>=quant_california[0.05]) & (california.MedianHouseValue<=quant_california[0.95] )]

In [ ]:
california.shape[0] - temp.shape[0]

2063

In [ ]:
california = temp

In [ ]:
california_scaler = MinMaxScaler()

In [ ]:
data = california_scaler.fit_transform(california.iloc[:, 2:-1])
target = np.asarray(california.MedianHouseValue)

data = data.astype(np.float32)
target = target.astype(np.float32)

california = {'data': torch.from_numpy(data), 'target': torch.from_numpy(target).unsqueeze(1)}

In [ ]:
california_dataset = torch.utils.data.TensorDataset(california["data"],california["target"])

In [ ]:
#hyperparameters
input_size = california["data"].shape[1]
hidden_size = [40,20]
num_classes = 1
num_epochs = 10
batch_size = 16
learning_rate = 0.001

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=california_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True,
                                           drop_last=True)

In [ ]:
model_california = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_california.parameters(), lr=learning_rate)  

In [ ]:
print(model_california)

NeuralNet(
  (fc1): Linear(in_features=6, out_features=40, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=40, out_features=20, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=20, out_features=1, bias=True)
)


In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        features = features.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_california(features)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 1000 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/10], Step [1000/1161], Loss: 7541.6885
Epoch [2/10], Step [1000/1161], Loss: 8244.5107
Epoch [3/10], Step [1000/1161], Loss: 3365.1958
Epoch [4/10], Step [1000/1161], Loss: 3144.7168
Epoch [5/10], Step [1000/1161], Loss: 3814.4595
Epoch [6/10], Step [1000/1161], Loss: 7797.0293
Epoch [7/10], Step [1000/1161], Loss: 2364.5972
Epoch [8/10], Step [1000/1161], Loss: 4998.7979
Epoch [9/10], Step [1000/1161], Loss: 4985.9502
Epoch [10/10], Step [1000/1161], Loss: 5098.6045


In [ ]:
input_dim =  california["data"].shape[1]
output_classes = 1
learning_rate = 0.001
batch_size = 16
num_epochs = 10
####
train_loader = torch.utils.data.DataLoader(dataset=california_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True,
                                           drop_last=True)


In [ ]:
model_california1 = DenseNeuralNet(input_dim, output_classes,layers=[50,100,50]).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_california1.parameters(), lr=learning_rate)  

In [ ]:
print(model_california1)

DenseNeuralNet(
  (net): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=50, bias=True)
    (5): ReLU()
    (6): Linear(in_features=50, out_features=1, bias=True)
  )
)


In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        features = features.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_california1(features)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 1000 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/10], Step [1000/1161], Loss: 3134.2261
Epoch [2/10], Step [1000/1161], Loss: 3744.9575
Epoch [3/10], Step [1000/1161], Loss: 7409.7461
Epoch [4/10], Step [1000/1161], Loss: 7529.2212
Epoch [5/10], Step [1000/1161], Loss: 2457.1008
Epoch [6/10], Step [1000/1161], Loss: 4122.2329
Epoch [7/10], Step [1000/1161], Loss: 2505.1204
Epoch [8/10], Step [1000/1161], Loss: 3388.9729
Epoch [9/10], Step [1000/1161], Loss: 8651.4434
Epoch [10/10], Step [1000/1161], Loss: 4482.0957
